<a href="https://colab.research.google.com/github/techburnvarun/groq-conversation-assignment/blob/main/AssignTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install openai requests

In [39]:
import json
import re
from typing import List, Dict, Any
from datetime import datetime
import openai
from openai import OpenAI

print("Starting Groq API Assignment...")
print("=" * 60)

Starting Groq API Assignment...


In [46]:
GROQ_API_KEY = "" # Put your real API key here!
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [41]:
# Create our client
client = OpenAI(
    api_key=GROQ_API_KEY,
    base_url=GROQ_BASE_URL
)

MODEL = "llama-3.1-8b-instant"

print("✅ Connected to Groq API successfully!")


✅ Connected to Groq API successfully!


 TASK 1: Managing Conversations with Smart Summarization

In [42]:

class ChatManager:
    """
    This class handles our conversation history and keeps it organized.
    It can summarize long conversations and manage memory efficiently.
    """

    def __init__(self):
        self.messages = []  # Store all our chat messages
        self.summary = ""   # Keep a running summary of old conversations
        self.message_count = 0
        print("Chat Manager initialized - ready to track conversations!")

    def add_chat_message(self, who_spoke, what_they_said):
        """Add a new message to our conversation history"""
        message = {
            "speaker": who_spoke,
            "message": what_they_said,
            "when": datetime.now().strftime("%H:%M:%S")
        }
        self.messages.append(message)
        self.message_count += 1

        print(f"Added message #{self.message_count} from {who_spoke}")

    def get_recent_messages(self, how_many):
        """Get the last few messages - useful for staying focused on recent topics"""
        if how_many <= 0:
            return self.messages

        recent = self.messages[-how_many:]
        print(f"Retrieved last {len(recent)} messages")
        return recent

    def get_messages_within_limit(self, character_limit):
        """Get messages that fit within a character limit, starting from most recent"""
        if character_limit <= 0:
            return self.messages

        selected_messages = []
        total_chars = 0

        # Work backwards from newest messages
        for message in reversed(self.messages):
            message_length = len(message["message"])

            if total_chars + message_length <= character_limit:
                selected_messages.insert(0, message)  # Add to beginning
                total_chars += message_length
            else:
                break

        print(f"Found {len(selected_messages)} messages within {character_limit} characters")
        return selected_messages

    def create_summary(self, messages_to_summarize):
        """Use Groq to create a smart summary of our conversation"""
        if not messages_to_summarize:
            return "Nothing to summarize yet."

        # Format the conversation nicely for the AI
        conversation_text = ""
        for msg in messages_to_summarize:
            conversation_text += f"{msg['speaker']}: {msg['message']}\n"

        print("Asking Groq to summarize the conversation...")

        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {
                        "role": "system",
                        "content": "You're great at summarizing conversations. Keep the main points and important details, but make it concise."
                    },
                    {
                        "role": "user",
                        "content": f"Please summarize this conversation:\n\n{conversation_text}"
                    }
                ],
                temperature=0.2,  # Keep it focused and consistent
                max_tokens=250
            )

            summary = response.choices[0].message.content.strip()
            print("Summary created successfully!")
            return summary

        except Exception as error:
            print(f"Oops, something went wrong: {error}")
            return f"Couldn't create summary: {error}"

    def check_if_time_to_summarize(self, every_n_messages):
        """Check if we should summarize after every N messages"""
        should_summarize = (self.message_count % every_n_messages == 0 and self.message_count > 0)

        if should_summarize:
            print(f"\n Time for periodic summarization! (After {self.message_count} messages)")

            # Create summary of current conversation
            new_summary = self.create_summary(self.messages)

            # If we already have a summary, combine them intelligently
            if self.summary:
                print("Combining with previous summary...")
                combined_content = f"Previous: {self.summary}\n\nRecent: {new_summary}"

                # Ask Groq to merge the summaries
                try:
                    response = client.chat.completions.create(
                        model=MODEL,
                        messages=[{
                            "role": "user",
                            "content": f"Combine these conversation summaries into one cohesive summary:\n{combined_content}"
                        }],
                        temperature=0.2,
                        max_tokens=300
                    )
                    self.summary = response.choices[0].message.content.strip()
                except:
                    self.summary = new_summary  # Fallback if combining fails
            else:
                self.summary = new_summary

            # Keep only the most recent messages, archive the rest in summary
            self.messages = self.messages[-3:]  # Keep last 3 messages
            print(" Conversation summarized and history cleaned up!")

        return should_summarize

    def show_current_status(self):
        """Display what's currently in our conversation manager"""
        print(f"\n Current Status:")
        print(f"    Total messages handled: {self.message_count}")
        print(f"    Messages in active memory: {len(self.messages)}")
        print(f"    Have summary: {'Yes' if self.summary else 'No'}")

        if self.summary:
            print(f"    Summary preview: {self.summary[:80]}...")

TASK 2: Smart Information Extraction from Chats

In [43]:
# Define what information we want to extract
INFO_EXTRACTION_FUNCTION = {
    "name": "extract_person_details",
    "description": "Pull out personal information mentioned in chat conversations",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "Person's full name if mentioned"
            },
            "email": {
                "type": "string",
                "description": "Email address if provided"
            },
            "phone": {
                "type": "string",
                "description": "Phone number in any format"
            },
            "location": {
                "type": "string",
                "description": "Where they live or are located"
            },
            "age": {
                "type": "integer",
                "description": "How old they are"
            }
        },
        "required": []  # Nothing is required - only extract what's actually mentioned
    }
}

class InfoExtractor:
    """
    This class is really good at finding personal information in chat conversations.
    It uses Groq's function calling to extract structured data.
    """

    def __init__(self):
        print(" Information Extractor ready to find details in conversations!")

    def extract_info_from_chat(self, chat_conversation):
        """
        Take a chat conversation and pull out any personal information mentioned
        """
        print(" Scanning conversation for personal information...")

        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {
                        "role": "system",
                        "content": "You're excellent at finding personal information in conversations. "
                                 "Only extract information that's clearly stated - don't guess or assume anything. "
                                 "If someone's name is mentioned, grab it. Same for email, phone, location, age."
                    },
                    {
                        "role": "user",
                        "content": f"Find any personal information in this chat:\n\n{chat_conversation}"
                    }
                ],
                functions=[INFO_EXTRACTION_FUNCTION],
                function_call={"name": "extract_person_details"},
                temperature=0.1  # Be very precise, not creative
            )

            # Parse the function call result
            if response.choices[0].message.function_call:
                extracted_data = json.loads(response.choices[0].message.function_call.arguments)
                print(" Information extraction completed!")
                return extracted_data
            else:
                print(" No structured information found")
                return {}

        except Exception as error:
            print(f" Extraction failed: {error}")
            return {"error": str(error)}

    def validate_extracted_info(self, extracted_info):
        """
        Check if the extracted information makes sense and follows our schema
        """
        print(" Validating extracted information...")

        validation_results = {
            "valid": True,
            "issues": [],
            "clean_data": {}
        }

        # Check each piece of information
        for field, value in extracted_info.items():
            if field == "name" and isinstance(value, str) and value.strip():
                validation_results["clean_data"]["name"] = value.strip()

            elif field == "email" and isinstance(value, str):
                # Simple email validation
                if "@" in value and "." in value:
                    validation_results["clean_data"]["email"] = value.strip()
                else:
                    validation_results["issues"].append(f"Email looks suspicious: {value}")

            elif field == "phone" and isinstance(value, str):
                # Phone numbers can be in many formats, just check if it has digits
                if any(char.isdigit() for char in value):
                    validation_results["clean_data"]["phone"] = value.strip()
                else:
                    validation_results["issues"].append(f"Phone number seems odd: {value}")

            elif field == "location" and isinstance(value, str) and value.strip():
                validation_results["clean_data"]["location"] = value.strip()

            elif field == "age":
                # Age should be a reasonable number
                try:
                    age_num = int(value) if isinstance(value, str) else value
                    if 0 < age_num < 150:  # Reasonable age range
                        validation_results["clean_data"]["age"] = age_num
                    else:
                        validation_results["issues"].append(f"Age seems unrealistic: {age_num}")
                except:
                    validation_results["issues"].append(f"Age is not a valid number: {value}")

        if validation_results["issues"]:
            validation_results["valid"] = False

        print(f" Validation complete - found {len(validation_results['issues'])} issues")
        return validation_results

DEMONSTRATION

In [44]:
def demo_conversation_management():
    """Show off how our conversation management works"""
    print("\n" + "="*60)
    print("🚀 DEMONSTRATING TASK 1: CONVERSATION MANAGEMENT")
    print("="*60)

    # Create our chat manager
    chat_manager = ChatManager()

    # Simulate a realistic conversation about coding help
    sample_chats = [
        ("user", "Hey, I'm struggling with my Python project and need some help"),
        ("assistant", "I'd be happy to help! What kind of Python project are you working on?"),
        ("user", "I'm building a web scraper to collect product prices from e-commerce sites"),
        ("assistant", "That sounds interesting! Web scraping can be tricky. What issues are you running into?"),
        ("user", "The main problem is that some sites are blocking my requests, even with basic headers"),
        ("assistant", "That's common. Try rotating user agents, adding random delays, and using session objects"),
        ("user", "Good idea! Should I also use proxies?"),
        ("assistant", "Proxies can help, but start with the simpler solutions first. Also consider using requests-html"),
        ("user", "Thanks! One more question - how should I store all this scraped data?"),
        ("assistant", "Depends on your needs. CSV files for simple data, SQLite for more complex relationships"),
    ]

    print("\n Adding messages to our conversation...")

    # Add each message and check for summarization every 4 messages
    for i, (speaker, message) in enumerate(sample_chats):
        chat_manager.add_chat_message(speaker, message)

        # Check if it's time to summarize (every 4 messages)
        if chat_manager.check_if_time_to_summarize(4):
            chat_manager.show_current_status()
            print()

    print("\n Testing different ways to retrieve conversation history:")

    # Test getting recent messages
    print("\n1 Getting last 3 messages:")
    recent_messages = chat_manager.get_recent_messages(3)
    for msg in recent_messages:
        print(f"   [{msg['when']}] {msg['speaker']}: {msg['message'][:50]}...")

    # Test getting messages within character limit
    print("\n2 Getting messages within 300 character limit:")
    limited_messages = chat_manager.get_messages_within_limit(300)
    total_chars = sum(len(msg['message']) for msg in limited_messages)
    print(f"   Found {len(limited_messages)} messages using {total_chars} characters")
    for msg in limited_messages:
        print(f"   [{msg['when']}] {msg['speaker']}: {msg['message'][:40]}...")

    # Show the summary if we have one
    if chat_manager.summary:
        print(f"\n Generated Summary:\n{chat_manager.summary}")

    chat_manager.show_current_status()

def demo_information_extraction():
    """Show off our information extraction capabilities"""
    print("\n" + "="*60)
    print(" DEMONSTRATING TASK 2: INFORMATION EXTRACTION")
    print("="*60)

    # Create our information extractor
    extractor = InfoExtractor()

    # Create realistic chat samples with different types of information
    test_chats = [
        {
            "title": "Customer Support Chat",
            "content": """
Support: Hi there! I'm here to help with your account issue.
Customer: Thanks! My name is Jennifer Martinez and I'm having trouble logging in.
Support: I can help with that Jennifer. Can I get your email address?
Customer: Sure, it's jennifer.martinez@email.com
Support: Great! And what's a good phone number to reach you?
Customer: You can call me at (555) 123-4567. I'm 29 years old, by the way.
Support: Perfect! And where are you located?
Customer: I'm in San Francisco, California.
            """
        },
        {
            "title": "Service Registration",
            "content": """
Rep: Welcome! Let's get you signed up for our premium service.
User: Sounds good! I'm David Kim, 34 years old.
Rep: Nice to meet you David! What's your contact information?
User: My email is d.kim@company.org and I live in Seattle, WA
Rep: Excellent! Any phone number?
User: Yes, my cell is 555-987-6543
Rep: Perfect, we'll get you all set up!
            """
        },
        {
            "title": "Consultation Booking",
            "content": """
Scheduler: I'd like to schedule a consultation for you.
Client: Great! I'm Sarah Johnson and I'm interested in your services.
Scheduler: Wonderful Sarah! How old are you, if you don't mind me asking?
Client: I'm 41. My email is sarah.j.johnson@domain.com
Scheduler: And your phone number?
Client: It's (555) 456-7890. I'm calling from Portland, Oregon.
Scheduler: Perfect! Let me get this scheduled for you.
            """
        }
    ]

    print(f"\n🧪 Processing {len(test_chats)} different chat conversations...\n")

    # Process each chat sample
    for i, chat_sample in enumerate(test_chats, 1):
        print(f"{'='*25} CHAT EXAMPLE {i} {'='*25}")
        print(f" {chat_sample['title']}")
        print(f" Conversation:\n{chat_sample['content'].strip()}\n")

        # Extract information
        extracted_info = extractor.extract_info_from_chat(chat_sample['content'])

        if "error" in extracted_info:
            print(f" Something went wrong: {extracted_info['error']}\n")
            continue

        # Show what we found
        print(" Information Found:")
        if extracted_info:
            for key, value in extracted_info.items():
                print(f"    {key.title()}: {value}")
        else:
            print("    No information extracted")

        # Validate the extracted information
        validation = extractor.validate_extracted_info(extracted_info)

        print(f"\n Validation Results:")
        if validation["valid"]:
            print("    All information looks good!")
            print("    Validated Data:")
            for key, value in validation["clean_data"].items():
                print(f"      ✓ {key}: {value}")
        else:
            print("    Found some issues:")
            for issue in validation["issues"]:
                print(f"      • {issue}")
            print("    Valid Data:")
            for key, value in validation["clean_data"].items():
                print(f"      ✓ {key}: {value}")

        print("\n")

def run_complete_demo():
    """Run the full demonstration of both tasks"""
    print(" STARTING COMPLETE GROQ API ASSIGNMENT DEMONSTRATION")
    print(" Started at:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(" Using model:", MODEL)

    try:
        # Quick connection test
        print("\n Testing connection to Groq API...")
        test_response = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": "Hi, just testing the connection!"}],
            max_tokens=5
        )
        print(" Connection successful!")

        # Run both demonstrations
        demo_conversation_management()
        demo_information_extraction()

        # Final summary
        print("\n" + "="*60)
        print(" ASSIGNMENT COMPLETED SUCCESSFULLY!")
        print("="*60)
        print("\n What we accomplished:")
        print(" Task 1 - Conversation Management:")
        print("   • ✓ Maintained running conversation history")
        print("   • ✓ Implemented truncation by message count and character limit")
        print("   • ✓ Added periodic summarization every N messages")
        print("   • ✓ Demonstrated with realistic conversation samples")
        print("\n Task 2 - Information Extraction:")
        print("   • ✓ Created JSON schema for extracting 5 types of personal info")
        print("   • ✓ Used Groq's function calling for structured extraction")
        print("   • ✓ Validated extracted data against our schema")
        print("   • ✓ Processed multiple realistic chat examples")
        print("\n Ready for GitHub submission!")

    except Exception as error:
        print(f"\n Something went wrong during the demo: {error}")
        print(" Make sure your Groq API key is correct and you have internet access")


LET'S RUN EVERYTHING!

In [45]:

if __name__ == "__main__":
    run_complete_demo()


 STARTING COMPLETE GROQ API ASSIGNMENT DEMONSTRATION
 Started at: 2025-09-15 04:32:51
 Using model: llama-3.1-8b-instant

 Testing connection to Groq API...
 Connection successful!

🚀 DEMONSTRATING TASK 1: CONVERSATION MANAGEMENT
Chat Manager initialized - ready to track conversations!

 Adding messages to our conversation...
Added message #1 from user
Added message #2 from assistant
Added message #3 from user
Added message #4 from assistant

 Time for periodic summarization! (After 4 messages)
Asking Groq to summarize the conversation...
Summary created successfully!
 Conversation summarized and history cleaned up!

 Current Status:
    Total messages handled: 4
    Messages in active memory: 3
    Have summary: Yes
    Summary preview: Here's a summary of the conversation:

The user is working on a Python project, ...

Added message #5 from user
Added message #6 from assistant
Added message #7 from user
Added message #8 from assistant

 Time for periodic summarization! (After 8 messa